In [1]:
import re

import pandas as pd

In [17]:
from tqdm import tqdm
import json
import pandas as pd
import re 
from difflib import SequenceMatcher
from collections import defaultdict
df = pd.read_excel("726四分类法.xlsx")


def similarity(sent1, sent2):
    return SequenceMatcher(None, sent1, sent2).ratio()


def split_sentences(data):
    split_data = []
    for item in data:
        # 使用正则表达式分割句子
        sentences = re.split(r'[？。]', item['content'])
        for sentence in sentences:
            if sentence:  # 忽略空字符串
                split_data.append({'type': item['type'], 'content': sentence})
    return split_data

gpt4o_output = []
manual_label = []
for i in tqdm(range(0,len(df))):
    # print(i)
    gpt4o = json.loads(df.loc[i,'predict']).get("result")
    # print(f'gpt4o: {gpt4o}')
    manual = json.loads(df.loc[i,'label'])
    # print(f'manual: {manual}')
    #分句
    split_data_gpt4o_output = split_sentences(gpt4o)
    # print(f"split_data_gpt4o_output: {split_data_gpt4o_output}")

    split_data_manual = split_sentences(manual)
    # print(f"split_data_manual: {split_data_manual}")
    # if len(split_data_gpt4o_output) != len(split_data_manual):
    #     print(i)
    #     # print(f"split_data_gpt4o_output: {len(split_data_gpt4o_output)}")
    #     # print(f"split_data_manual: {len(split_data_manual)}")
    #     # print(f"split_data_gpt4o_output: {split_data_gpt4o_output}")
    #     # print(f"split_data_manual: {split_data_manual}")
    #     # break
    for item in split_data_gpt4o_output:
        for sent in split_data_manual:
            
            if similarity(item.get("content") ,sent.get("content")) >0.5:
                item['label'] = sent.get("type")
                item['manual_content'] = sent.get("content")
                break  # 如果匹配上了，就不要再往下匹配了。

    gpt4o_output.extend(split_data_gpt4o_output)
# 
    # manual_label.extend(split_data_manual)

100%|██████████| 11/11 [00:00<00:00, 117.53it/s]


In [18]:
# 定义一个包含四个必要键值的列表
required_keys = ['type', 'content', 'label', 'manual_content']

# 遍历每个字典
# data = [
#     {'type': '讲解', 'content': '猴子呀他一开始很勤劳啊，还是种了一棵梨树', 'label': '讲解', 'manual_content': '猴子呀他一开始很勤劳啊，还是种了一棵梨树'},
#     {'type': '讲解', 'content': '可是当乌鸦来劝说它的时候，它就拔掉了梨树，改种了', 'label': '讲解', 'manual_content': '可是当乌鸦来劝说它的时候，它就拔掉了梨树，改种了'},
#     {'type': '讲解', 'content': '正当杏树成活的时候，喜鹊来劝他了', 'label': '讲解', 'manual_content': '正当杏树成活的时候，喜鹊来劝他了'}
# ]

# 遍历列表并确保每个字典都有四个键值
for item in gpt4o_output:
    for key in required_keys:
        if key not in item:
            item[key] = ''  # 如果缺少键，就添加并赋值为空字符串

# 打印结果
print(gpt4o_output)


[{'type': '发起', 'content': '请袁艺喊“开始上课”，声音要响亮啊', 'label': '发起', 'manual_content': '请袁艺喊“开始上课”，声音要响亮啊'}, {'type': '其它', 'content': '同学们好，请坐', 'label': '发起', 'manual_content': '同学们好，请坐'}, {'type': '评价', 'content': '真棒！', 'label': '评价', 'manual_content': '真棒'}, {'type': '讲解', 'content': '昨天呀，我们一起学习了课文八', 'label': '讲解', 'manual_content': '昨天呀，我们一起学习了课文八'}, {'type': '讲解', 'content': '今天小猴子呀，要到我们班来做客了', 'label': '讲解', 'manual_content': '今天，小猴子呀要到我们班来做客了'}, {'type': '发起', 'content': '我们来跟他打打招呼', 'label': '发起', 'manual_content': '我们来跟他打打招呼'}, {'type': '讲解', 'content': '猴子，昨天在写作业的时候呀，小朋友要注意哦', 'label': '讲解', 'manual_content': '昨天在写作业的时候呀，小朋友要注意哦'}, {'type': '讲解', 'content': '这个“猴”，反犬旁，旁边是一个单立人', 'label': '讲解', 'manual_content': '这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊'}, {'type': '讲解', 'content': '中间有没有一个短竖啊', 'label': '讲解', 'manual_content': '这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊'}, {'type': '讲解', 'content': '那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了', 'label': '讲解', 'manual_content': '那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字

In [19]:
# 将数据转换为DataFrame
df = pd.DataFrame(gpt4o_output)
df

,type,content,label,manual_content
0,发起,请袁艺喊“开始上课”，声音要响亮啊,发起,请袁艺喊“开始上课”，声音要响亮啊
1,其它,同学们好，请坐,发起,同学们好，请坐
2,评价,真棒！,评价,真棒
3,讲解,昨天呀，我们一起学习了课文八,讲解,昨天呀，我们一起学习了课文八
4,讲解,今天小猴子呀，要到我们班来做客了,讲解,今天，小猴子呀要到我们班来做客了
...,...,...,...,...
404,评价,很好！,评价,很好！
405,发起,好，再来一个同学，李春月，你来,发起,好，再来一个同学，李春月，你来
406,发起,你说，“改革”,讲解,你说，“改革”
407,发起,你说，我们学下一个字，“洗”，预备起,发起,你说，我们学下一个字，“洗”，预备起


In [27]:
df = df[(df != '').all(axis=1)]
df_cleaned = df.dropna(subset='label').reset_index(drop=True)
df_cleaned

,type,content,label,manual_content
0,发起,请袁艺喊“开始上课”，声音要响亮啊,发起,请袁艺喊“开始上课”，声音要响亮啊
1,其它,同学们好，请坐,发起,同学们好，请坐
2,评价,真棒！,评价,真棒
3,讲解,昨天呀，我们一起学习了课文八,讲解,昨天呀，我们一起学习了课文八
4,讲解,今天小猴子呀，要到我们班来做客了,讲解,今天，小猴子呀要到我们班来做客了
...,...,...,...,...
401,评价,很好！,评价,很好！
402,发起,好，再来一个同学，李春月，你来,发起,好，再来一个同学，李春月，你来
403,发起,你说，“改革”,讲解,你说，“改革”
404,发起,你说，我们学下一个字，“洗”，预备起,发起,你说，我们学下一个字，“洗”，预备起


In [28]:
from sklearn.metrics import classification_report

df_cleaned['type'] = df_cleaned['type'].replace("其他","其它")
df_cleaned['label'] = df_cleaned['label'].replace("其他","其它")
# 计算分类报告
classification_rep = classification_report(df_cleaned['label'], df_cleaned['type'], zero_division=0)

print(classification_rep)

              precision    recall  f1-score   support

          其它       0.20      0.41      0.27        22
          发起       0.78      0.78      0.78       179
          讲解       0.58      0.75      0.65       104
          评价       0.87      0.41      0.55       101

    accuracy                           0.66       406
   macro avg       0.61      0.59      0.56       406
weighted avg       0.72      0.66      0.66       406



In [29]:
df_cleaned.to_excel("gpt4o_T_4_result.xlsx",index=False)

In [3]:
# 求取两个列表的交集和差集
content_manual = {item['content'] for item in split_data_manual}
content_gpt4o_output = {item['content'] for item in split_data_gpt4o_output}

# 交集
intersection = content_manual.intersection(content_gpt4o_output)

# 差集
difference_manual = content_manual.difference(content_gpt4o_output)
difference_gpt4o_output = content_gpt4o_output.difference(content_manual)

intersection, difference_manual, difference_gpt4o_output

({'他每天都去浇，天天都去浇水施肥',
  '你为什么要连喊两声“猴哥”呀',
  '你为什么还连喊两声呀',
  '你们找到了动作了吗',
  '你们看，这个猴子种树啊，它有一个动作的过程',
  '你觉得猴哥可能会听不见，是不是啊',
  '再请一只小乌鸦，好，你来小乌鸦',
  '叫一声就可以了呀',
  '哎，小乌鸦你等等啊',
  '哦，这只小乌鸦读的真好，读的真顺畅啊',
  '唉，没有提示你还能不能把刚刚的故事开头讲给我们听',
  '嗯，施肥',
  '嗯，细心',
  '好熟练啊，把故事啊，这么大声的就告诉我们了',
  '好，乌鸦哇哇的对猴子说了什么呢',
  '好，你来',
  '好，我们按照这个顺序啊，把故事的开头自己在下面讲一讲，好不好',
  '好，谁来把故事的开头讲给我们听',
  '好，非常好',
  '对，非常勤劳',
  '然后呢',
  '然后等着将来可以吃梨子了',
  '猴子是不是很勤劳',
  '现在就直接浇水了吗',
  '而且我看出来了，你整的口水都快流出来了，是不是',
  '董新磊，',
  '说明他怎么样',
  '谁来做一下乌鸦',
  '还有呢',
  '这时候谁来',
  '那接下来我要加大难度了',
  '非常好'},
 {'乌鸦来', '你说，', '刘若涵，', '哇，这只猴子呀，真勤劳', '张文婷，', '来', '棒不棒'},
 {'你说，乌鸦来', '刘若涵，哇，这只猴子呀，真勤劳', '张文婷，棒不棒'})

In [4]:
split_data_gpt4o_output

[{'type': '评价', 'content': '嗯，细心'},
 {'type': '发起', 'content': '还有呢'},
 {'type': '讲解', 'content': '他每天都去浇，天天都去浇水施肥'},
 {'type': '讲解', 'content': '说明他怎么样'},
 {'type': '评价', 'content': '对，非常勤劳'},
 {'type': '评价', 'content': '猴子是不是很勤劳'},
 {'type': '讲解', 'content': '你们看，这个猴子种树啊，它有一个动作的过程'},
 {'type': '讲解', 'content': '你们找到了动作了吗'},
 {'type': '讲解', 'content': '好，你来'},
 {'type': '讲解', 'content': '现在就直接浇水了吗'},
 {'type': '讲解', 'content': '然后呢'},
 {'type': '讲解', 'content': '嗯，施肥'},
 {'type': '讲解', 'content': '然后等着将来可以吃梨子了'},
 {'type': '讲解', 'content': '好，我们按照这个顺序啊，把故事的开头自己在下面讲一讲，好不好'},
 {'type': '发起', 'content': '好，谁来把故事的开头讲给我们听'},
 {'type': '发起', 'content': '刘若涵，哇，这只猴子呀，真勤劳'},
 {'type': '发起', 'content': '而且我看出来了，你整的口水都快流出来了，是不是'},
 {'type': '评价', 'content': '好，非常好'},
 {'type': '讲解', 'content': '那接下来我要加大难度了'},
 {'type': '讲解', 'content': '唉，没有提示你还能不能把刚刚的故事开头讲给我们听'},
 {'type': '发起', 'content': '张文婷，棒不棒'},
 {'type': '发起', 'content': '好熟练啊，把故事啊，这么大声的就告诉我们了'},
 {'type': '发起', 'content': '非常好'},
 {'typ

In [5]:
split_data_manual

[{'type': '评价', 'content': '嗯，细心'},
 {'type': '发起', 'content': '还有呢'},
 {'type': '发起', 'content': '他每天都去浇，天天都去浇水施肥'},
 {'type': '发起', 'content': '说明他怎么样'},
 {'type': '评价', 'content': '对，非常勤劳'},
 {'type': '评价', 'content': '猴子是不是很勤劳'},
 {'type': '讲解', 'content': '你们看，这个猴子种树啊，它有一个动作的过程'},
 {'type': '发起', 'content': '你们找到了动作了吗'},
 {'type': '发起', 'content': '好，你来'},
 {'type': '发起', 'content': '现在就直接浇水了吗'},
 {'type': '发起', 'content': '然后呢'},
 {'type': '评价', 'content': '嗯，施肥'},
 {'type': '讲解', 'content': '然后等着将来可以吃梨子了'},
 {'type': '发起', 'content': '好，我们按照这个顺序啊，把故事的开头自己在下面讲一讲，好不好'},
 {'type': '发起', 'content': '来'},
 {'type': '发起', 'content': '好，谁来把故事的开头讲给我们听'},
 {'type': '发起', 'content': '刘若涵，'},
 {'type': '评价', 'content': '哇，这只猴子呀，真勤劳'},
 {'type': '评价', 'content': '而且我看出来了，你整的口水都快流出来了，是不是'},
 {'type': '评价', 'content': '好，非常好'},
 {'type': '发起', 'content': '那接下来我要加大难度了'},
 {'type': '发起', 'content': '唉，没有提示你还能不能把刚刚的故事开头讲给我们听'},
 {'type': '发起', 'content': '张文婷，'},
 {'type': '评价', 'content': '棒不棒'

In [10]:

from difflib import SequenceMatcher
from collections import defaultdict
text1 = "你说，乌鸦来"
text2 = "乌鸦来"
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()
similarity(text1,text2)

0.6666666666666666

In [7]:
for item in split_data_gpt4o_output:
    for sent in split_data_manual:
        
        if similarity(item.get("content") ,sent.get("content")) >0.8:
            item['label'] = sent.get("type")
            break  # 如果匹配上了，就不要再往下匹配了。


In [8]:
split_data_gpt4o_output

[{'type': '评价', 'content': '嗯，细心', 'label': '评价'},
 {'type': '发起', 'content': '还有呢', 'label': '发起'},
 {'type': '讲解', 'content': '他每天都去浇，天天都去浇水施肥', 'label': '发起'},
 {'type': '讲解', 'content': '说明他怎么样', 'label': '发起'},
 {'type': '评价', 'content': '对，非常勤劳', 'label': '评价'},
 {'type': '评价', 'content': '猴子是不是很勤劳', 'label': '评价'},
 {'type': '讲解', 'content': '你们看，这个猴子种树啊，它有一个动作的过程', 'label': '讲解'},
 {'type': '讲解', 'content': '你们找到了动作了吗', 'label': '发起'},
 {'type': '讲解', 'content': '好，你来', 'label': '发起'},
 {'type': '讲解', 'content': '现在就直接浇水了吗', 'label': '发起'},
 {'type': '讲解', 'content': '然后呢', 'label': '发起'},
 {'type': '讲解', 'content': '嗯，施肥', 'label': '评价'},
 {'type': '讲解', 'content': '然后等着将来可以吃梨子了', 'label': '讲解'},
 {'type': '讲解', 'content': '好，我们按照这个顺序啊，把故事的开头自己在下面讲一讲，好不好', 'label': '发起'},
 {'type': '发起', 'content': '好，谁来把故事的开头讲给我们听', 'label': '发起'},
 {'type': '发起', 'content': '刘若涵，哇，这只猴子呀，真勤劳', 'label': '评价'},
 {'type': '发起', 'content': '而且我看出来了，你整的口水都快流出来了，是不是', 'label': '评价'},
 {'type': '评价'

In [9]:
manual

[{'type': '评价', 'content': '嗯，细心。'},
 {'type': '发起', 'content': '还有呢？他每天都去浇，天天都去浇水施肥。说明他怎么样？'},
 {'type': '评价', 'content': '对，非常勤劳。猴子是不是很勤劳？'},
 {'type': '讲解', 'content': '你们看，这个猴子种树啊，它有一个动作的过程。'},
 {'type': '发起', 'content': '你们找到了动作了吗？好，你来。'},
 {'type': '发起', 'content': '现在就直接浇水了吗？'},
 {'type': '发起', 'content': '然后呢？'},
 {'type': '评价', 'content': '嗯，施肥。'},
 {'type': '讲解', 'content': '然后等着将来可以吃梨子了。'},
 {'type': '发起', 'content': '好，我们按照这个顺序啊，把故事的开头自己在下面讲一讲，好不好？'},
 {'type': '发起', 'content': '来。好，谁来把故事的开头讲给我们听？刘若涵，'},
 {'type': '评价', 'content': '哇，这只猴子呀，真勤劳。而且我看出来了，你整的口水都快流出来了，是不是？好，非常好。'},
 {'type': '发起', 'content': '那接下来我要加大难度了。唉，没有提示你还能不能把刚刚的故事开头讲给我们听？张文婷，'},
 {'type': '评价', 'content': '棒不棒？好熟练啊，把故事啊，这么大声的就告诉我们了。非常好。'},
 {'type': '发起', 'content': '这时候谁来？这时候谁来？你说，'},
 {'type': '讲解', 'content': '乌鸦来。'},
 {'type': '发起', 'content': '好，乌鸦哇哇的对猴子说了什么呢？谁来做一下乌鸦？董新磊，'},
 {'type': '评价', 'content': '哦，这只小乌鸦读的真好，读的真顺畅啊。'},
 {'type': '发起', 'content': '再请一只小乌鸦，好，你来小乌鸦。'},
 {'type': '其它', 'content': 

1


In [7]:
if 
# 分句处理
# split_data = split_sentences(data)

In [8]:
# split_data_gpt4o_output = split_sentences(gpt4o_output)
# split_data_gpt4o_output

In [9]:
# split_data_manual = split_sentences(manual_label)
# split_data_manual

In [10]:
# len(split_data_manual)


In [11]:
# len(split_data_gpt4o_output)

In [12]:
import pandas as pd 
content = [item.get("content") for item in split_data_manual]
label = [item.get("type") for item in split_data_manual]
preds = [item.get("type") for item in split_data_gpt4o_output]


In [ ]:
df = pd.DataFrame()
df['content'] = content
df['label'] = label
df['preds'] = preds
df

In [13]:
# import pandas as pd
# from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# 
# # 读取Excel文件
# # def calculate_metrics_from_excel(df):
# #     # 读取Excel文件
# #    
# # 
# #     # 假设Excel文件中有 'label' 和 'preds' 两列
# #     y_true = df['label']
# #     y_pred = df['preds']
# # 
# #     # 获取所有类型的类别标签
# #     types = list(set(y_true) | set(y_pred))
# # 
# #     # 计算每个类型的precision, recall, f1
# #     precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=types, zero_division=0)
# # 
# #     # 总体准确率
# #     accuracy = accuracy_score(y_true, y_pred)
# # 
# #     # 汇总每个类型的结果
# #     type_results = {}
# #     for t, p, r, f in zip(types, precision, recall, f1):
# #         type_results[t] = {
# #             "precision": p,
# #             "recall": r,
# #             "f1": f
# #         }
# # 
# #     # 计算总F1
# #     overall_f1 = sum(f1) / len(f1)
# # 
# #     return accuracy, overall_f1, type_results
# # 
# # accuracy, overall_f1, type_results = calculate_metrics_from_excel(df)
# # 
# # print(f"总体准确率: {accuracy:.2%}")
# # print(f"总体F1分数: {overall_f1:.2%}")
# # print("每个类型的结果:")
# # for t, result in type_results.items():
# #     print(f"类型: {t} - Precision: {result['precision']:.2%}, Recall: {result['recall']:.2%}, F1: {result['f1']:.2%}")


In [14]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from difflib import SequenceMatcher
from collections import defaultdict

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def evaluate_model(predictions, labels, threshold=0.99):
    y_true = []
    y_pred = []

    for pred, label in zip(predictions, labels):
        if similarity(pred['content'], label['content']) > threshold:
            y_true.append(label['type'])
            y_pred.append(pred['type'])
        else:
            y_true.append(label['type'])
            y_pred.append('不匹配')  # 如果内容不匹配，视为不匹配

    # 计算每个类型的precision, recall, f1
    types = list(set(y_true + y_pred))
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=types, zero_division=0)
    
    # 总体准确率
    accuracy = accuracy_score(y_true, y_pred)
    
    # 汇总每个类型的结果
    type_results = {}
    for t, p, r, f in zip(types, precision, recall, f1):
        type_results[t] = {
            "precision": p,
            "recall": r,
            "f1": f
        }

    # 计算总F1
    overall_f1 = sum(f1) / len(f1)

    return accuracy, overall_f1, type_results

# 模型预测的输出
predictions =split_data_gpt4o_output

# 人工标注的内容
labels = split_data_manual

accuracy, overall_f1, type_results = evaluate_model(predictions, labels)

print(f"总体准确率: {accuracy:.2%}")
print(f"总体F1分数: {overall_f1:.2%}")
print("每个类型的结果:")
for t, result in type_results.items():
    print(f"类型: {t} - Precision: {result['precision']:.2%}, Recall: {result['recall']:.2%}, F1: {result['f1']:.2%}")


总体准确率: 10.26%
总体F1分数: 16.03%
每个类型的结果:
类型: 不匹配 - Precision: 0.00%, Recall: 0.00%, F1: 0.00%
类型: 其它 - Precision: 33.33%, Recall: 100.00%, F1: 50.00%
类型: 评价 - Precision: 0.00%, Recall: 0.00%, F1: 0.00%
类型: 讲解 - Precision: 0.00%, Recall: 0.00%, F1: 0.00%
类型: 其他 - Precision: 0.00%, Recall: 0.00%, F1: 0.00%
类型: 发起 - Precision: 100.00%, Recall: 30.00%, F1: 46.15%


In [28]:

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()
a = "请袁艺喊“开始上课”，声音要响亮啊"
b = "请袁艺喊“开始上课”，声音要响亮啊"
similarity(a,b)

1.0

In [38]:
for item1,item2 in zip(split_data_gpt4o_output,split_data_manual):
    
    print(f'gpt4o输出：{item1.get("content")}\n人工标注的：{item2.get("content")}\n模型指标：{similarity(item1.get("content"),item2.get("content"))}')

gpt4o输出：请袁艺喊“开始上课”，声音要响亮啊
人工标注的：请袁艺喊“开始上课”，声音要响亮啊
模型指标：1.0
gpt4o输出：同学们好，请坐
人工标注的：同学们好，请坐
模型指标：1.0
gpt4o输出：真棒
人工标注的：真棒
模型指标：1.0
gpt4o输出：昨天呀，我们一起学习了课文八
人工标注的：昨天呀，我们一起学习了课文八
模型指标：1.0
gpt4o输出：今天，小猴子呀要到我们班来做客了
人工标注的：今天，小猴子呀要到我们班来做客了
模型指标：1.0
gpt4o输出：我们来跟他打打招呼
人工标注的：我们来跟他打打招呼
模型指标：1.0
gpt4o输出：“猴子”
人工标注的：“猴子”
模型指标：1.0
gpt4o输出：昨天在写作业的时候呀，小朋友要注意哦
人工标注的：昨天在写作业的时候呀，小朋友要注意哦
模型指标：1.0
gpt4o输出：这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊
人工标注的：这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊
模型指标：1.0
gpt4o输出：那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了
人工标注的：那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了
模型指标：1.0
gpt4o输出：明白了吗
人工标注的：明白了吗
模型指标：1.0
gpt4o输出：好，现在用眼睛看，用心记住这个字，“猴子”
人工标注的：好，现在用眼睛看，用心记住这个字，“猴子”
模型指标：1.0
gpt4o输出：哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树
人工标注的：哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树
模型指标：1.0
gpt4o输出：它分别种了什么树呢
人工标注的：它分别种了什么树呢
模型指标：1.0
gpt4o输出：谁来说说
人工标注的：谁来说说
模型指标：1.0
gpt4o输出：于凯，你来说说看
人工标注的：于凯，你来说说看
模型指标：1.0
gpt4o输出：你慢讲啊
人工标注的：你慢讲啊
模型指标：1.0
gpt4o输出：嗯，
人工标注的：嗯，
模型指标：1.0
gpt4o输出：然后呢
人工标注的：然后呢
模型指标：1.0
gpt4o输出：然后种了杏树
人工标注的：然后种了杏树
模型指标：1.0
gpt4o输出：最后呢
人工标注的：最后呢
模型